# Withdraw Unit Test
Performing verification of the withdraw mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

loaded GraphQL_data/SIstakeDelegateds.csv.
loaded GraphQL_data/SIstakeLockeds.csv.
loaded GraphQL_data/SIstakeWithdrawns.csv.
loaded GraphQL_data/SIallocationCollecteds.csv.
loaded GraphQL_data/SIstakeDepositeds.csv.


Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
withdraw_events_dict = withdraw_events[0]

print("EXPECTED TRUTH--Tokens withdrawn via withdraw events:")
for timestep, withdraw_event in withdraw_events_dict.items():
    print(f"{timestep-1}, {withdraw_event[0]['delegator']}, {withdraw_event[0]['tokens']}")



EXPECTED TRUTH--Tokens withdrawn via withdraw events:
75, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2984.999999999999999999
86, 0x40f657e85bbb5cc7beb402da315c3fe1974442bf, 1057.880681157946400522
91, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1163.668749273741000575
149, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3990.676349999999999999
193, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 60961.231577108159627625


In [20]:
print("MODELED RESULTS--Tokens locked in undelegation.")
for timestep, withdraw_event in withdraw_events_dict.items():
    # print(f"{timestep-1}") 
    # print(f"{undelegation_event[0]['delegator']}") 
    # print(f"{df.iloc[timestep-1].delegators[undelegation_event[0]['delegator']].shares}")
    tokens_withdrawn = df.iloc[timestep-1].delegators[withdraw_event[0]['delegator']].holdings - df.iloc[timestep-2].delegators[withdraw_event[0]['delegator']].holdings
    print(f"{timestep-1}, {withdraw_event[0]['delegator']}, {tokens_withdrawn=}")

MODELED RESULTS--Tokens locked in undelegation.
75, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, tokens_withdrawn=Decimal('0E-18')
86, 0x40f657e85bbb5cc7beb402da315c3fe1974442bf, tokens_withdrawn=Decimal('0E-18')
91, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, tokens_withdrawn=Decimal('1104.691161886919640222230789')
149, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, tokens_withdrawn=Decimal('3787.745417074846793187960997')
193, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, tokens_withdrawn=Decimal('55887.12136222974143499817302')


In [23]:
print("UNITTEST RESULTS")
for timestep, withdraw_event in withdraw_events_dict.items():
    tokens_withdrawn = df.iloc[timestep-1].delegators[withdraw_event[0]['delegator']].holdings - df.iloc[timestep-2].delegators[withdraw_event[0]['delegator']].holdings
    print(f"Event: Timestep={timestep-1}, Delegator ID={withdraw_event[0]['delegator']}, Tokens Withdrawn={withdraw_event[0]['tokens']}")
    print(f"Model: Timestep={timestep-1}, Delegator ID={withdraw_event[0]['delegator']}, Tokens Withdrawn={tokens_withdrawn}")
    print("Withdraw Amount Equal?", withdraw_event[0]['tokens'] == tokens_withdrawn)
    print()


UNITTEST RESULTS
Event: Timestep=75, Delegator ID=0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, Tokens Withdrawn=2984.999999999999999999
Model: Timestep=75, Delegator ID=0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, Tokens Withdrawn=0E-18
Withdraw Amount Equal? False

Event: Timestep=86, Delegator ID=0x40f657e85bbb5cc7beb402da315c3fe1974442bf, Tokens Withdrawn=1057.880681157946400522
Model: Timestep=86, Delegator ID=0x40f657e85bbb5cc7beb402da315c3fe1974442bf, Tokens Withdrawn=0E-18
Withdraw Amount Equal? False

Event: Timestep=91, Delegator ID=0x63f8168f665f79101a0e0da2313cd1391e1c5e27, Tokens Withdrawn=1163.668749273741000575
Model: Timestep=91, Delegator ID=0x63f8168f665f79101a0e0da2313cd1391e1c5e27, Tokens Withdrawn=1104.691161886919640222230789
Withdraw Amount Equal? False

Event: Timestep=149, Delegator ID=0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, Tokens Withdrawn=3990.676349999999999999
Model: Timestep=149, Delegator ID=0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, Tokens Withdrawn=3

# Conclusion
The number of tokens withdrawn, measured as (holdings_after_event) - (holdings_before_event), is 0 in the first two events, possibly due to the locked_until threshold not being met.  In the next three, we are close, but diverging again.